In [1]:
import requests
import tkinter as tk
from PIL import Image, ImageTk
import io

def get_pokemon_details(pokemon_name):
    url = f"https://pokeapi.co/api/v2/pokemon/{pokemon_name.lower()}"
    response = requests.get(url)
    if response.status_code == 200:
        pokemon_data = response.json()
        stats = {stat['stat']['name']: stat['base_stat'] for stat in pokemon_data['stats']}
        return {
            "name": pokemon_data["name"],
            "id": pokemon_data["id"],
            "height": pokemon_data["height"],
            "weight": pokemon_data["weight"],
            "types": [type_info["type"]["name"] for type_info in pokemon_data["types"]],
            "abilities": [ability_info["ability"]["name"] for ability_info in pokemon_data["abilities"]],
            "sprite": pokemon_data["sprites"]["front_default"],
            "hp": stats.get("hp"),
            "attack": stats.get("attack"),
            "defense": stats.get("defense"),
            "special-attack": stats.get("special-attack"),
            "special-defense": stats.get("special-defense"),
            "speed": stats.get("speed"),
        }
    else:
        return None

def calculate_damage(attacker, defender):
    # Simple damage calculation: (Attack - Defense) / 2
    attack = attacker['attack']
    defense = defender['defense']
    damage = max(0, (attack - defense) // 2)
    return damage

def battle(pokemon1, pokemon2):
    hp1, hp2 = pokemon1['hp'], pokemon2['hp']
    turn = 1
    results = []
    
    while hp1 > 0 and hp2 > 0:
        if turn % 2 != 0:  # Pokémon 1's turn
            damage = calculate_damage(pokemon1, pokemon2)
            hp2 -= damage
            results.append(f"{pokemon1['name']} attacks {pokemon2['name']} for {damage} damage!")
        else:  # Pokémon 2's turn
            damage = calculate_damage(pokemon2, pokemon1)
            hp1 -= damage
            results.append(f"{pokemon2['name']} attacks {pokemon1['name']} for {damage} damage!")
        
        results.append(f"{pokemon1['name']} HP: {hp1}")
        results.append(f"{pokemon2['name']} HP: {hp2}")
        
        turn += 1
    
    if hp1 <= 0 and hp2 <= 0:
        results.append("It's a tie!")
    elif hp1 <= 0:
        results.append(f"{pokemon2['name']} wins!")
    else:
        results.append(f"{pokemon1['name']} wins!")
    
    return results

def update_battle_log(results):
    for result in results:
        battle_log.insert(tk.END, result + '\n')
    battle_log.yview(tk.END)

def start_battle():
    pokemon_name1 = entry_pokemon1.get()
    pokemon_name2 = entry_pokemon2.get()

    global pokemon1, pokemon2
    pokemon1 = get_pokemon_details(pokemon_name1)
    pokemon2 = get_pokemon_details(pokemon_name2)

    if pokemon1 and pokemon2:
        update_battle_log([f"\n{pokemon1['name']} vs {pokemon2['name']}"])
        result = battle(pokemon1, pokemon2)
        update_battle_log(result)
        
        # Update Pokémon images
        image1 = Image.open(io.BytesIO(requests.get(pokemon1['sprite']).content))
        image1 = ImageTk.PhotoImage(image1)
        label_pokemon1_img.config(image=image1)
        label_pokemon1_img.image = image1
        
        image2 = Image.open(io.BytesIO(requests.get(pokemon2['sprite']).content))
        image2 = ImageTk.PhotoImage(image2)
        label_pokemon2_img.config(image=image2)
        label_pokemon2_img.image = image2
    else:
        update_battle_log([f"Pokémon '{pokemon_name1}' or '{pokemon_name2}' not found!"])

# Create the main application window
root = tk.Tk()
root.title("Pokémon Battle Simulator")

# Input fields
tk.Label(root, text="First Pokémon:").pack()
entry_pokemon1 = tk.Entry(root)
entry_pokemon1.pack()

tk.Label(root, text="Second Pokémon:").pack()
entry_pokemon2 = tk.Entry(root)
entry_pokemon2.pack()

# Start Battle button
tk.Button(root, text="Start Battle", command=start_battle).pack()

# Battle log
battle_log = tk.Text(root, height=15, width=50)
battle_log.pack()

# Pokémon images
frame_images = tk.Frame(root)
frame_images.pack()

label_pokemon1_img = tk.Label(frame_images)
label_pokemon1_img.pack(side=tk.LEFT)

label_pokemon2_img = tk.Label(frame_images)
label_pokemon2_img.pack(side=tk.RIGHT)

# Run the application
root.mainloop()


KeyboardInterrupt: 